**Задание 1:**

Создать базу, добавить в нее 4 таблицы с данными, и реализовать следующие действия:

*   HAVING
*   JOIN (LEFT, INNER)
*   UNION
*   подзапросы

In [436]:
import sqlalchemy as db
from sqlalchemy import text

In [437]:
engine = db.create_engine('sqlite:///library.db')
conn = engine.connect()
metadata = db.MetaData()

In [438]:
create_authors_table_query = """
CREATE TABLE authors (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL
)
"""

In [439]:
create_books_table_query = """
CREATE TABLE books (
    id INTEGER PRIMARY KEY,
    title TEXT NOT NULL,
    author_id INTEGER,
    price REAL
)
"""

In [440]:
create_categories_table_query = """
CREATE TABLE categories (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL
)
"""

In [441]:
create_book_categories_table_query = """
CREATE TABLE book_categories (
    book_id INTEGER,
    category_id INTEGER,
    PRIMARY KEY (book_id, category_id),
    FOREIGN KEY (book_id) REFERENCES books (id),
    FOREIGN KEY (category_id) REFERENCES categories (id)
)
"""

В чем отличия conn.exec_driver_sql() и conn.execute(text()) ?

In [433]:
conn.exec_driver_sql(create_authors_table_query)
conn.exec_driver_sql(create_books_table_query)
conn.exec_driver_sql(create_categories_table_query)
conn.exec_driver_sql(create_book_categories_table_query)

In [442]:
conn.execute(text(create_authors_table_query))
conn.execute(text(create_books_table_query))
conn.execute(text(create_categories_table_query))
conn.execute(text(create_book_categories_table_query))

In [443]:
insert_authors_query = """
INSERT INTO authors (name) VALUES
('Эдуард Успенский'),
('Лев Толстой'),
('Марк Твен'),
('Фёдор Достоевский'),
('Александр Пушкин');
"""

In [444]:
insert_books_query = """
INSERT INTO books (title, author_id, price) VALUES
('Чебурашка', 1, 15.0),
('Война и мир', 2, 30.0),
('Анна Каренина', 2, 25.0),
('Евгений Онегин', 5, 20.0),
('Преступление и наказание', 4, 22.0),
('Бесы', 4, 24.0),
('Приключения Тома Сойера', 3, 19.0);
"""

In [445]:
insert_categories_query = """
INSERT INTO categories (name) VALUES
('Детская литература'),
('Классическая литература'),
('Поэзия'),
('Роман');
"""

In [446]:
insert_book_categories_query = """
INSERT INTO book_categories (book_id, category_id) VALUES
(1, 1),
(2, 2),
(3, 2),
(4, 3),
(5, 2),
(6, 2),
(7, null);
"""

In [447]:
conn.execute(text(insert_authors_query))
conn.execute(text(insert_books_query))
conn.execute(text(insert_categories_query))
conn.execute(text(insert_book_categories_query))

In [448]:
# HAVING

query_having = """
SELECT authors.name, COUNT(books.id) AS book_count
FROM authors
JOIN books
ON authors.id = books.author_id
GROUP BY authors.name
HAVING COUNT(books.id) > 1
"""

In [449]:
result = conn.execute(text(query_having))
print("HAVING:")
for row in result:
    print(row)

HAVING:
('Лев Толстой', 2)
('Фёдор Достоевский', 2)


In [450]:
# JOIN (INNER)

query_inner_join = """
SELECT books.title, authors.name, categories.name
FROM books
INNER JOIN authors
ON books.author_id = authors.id
INNER JOIN book_categories
ON books.id = book_categories.book_id
INNER JOIN categories
ON book_categories.category_id = categories.id
"""

In [451]:
result = conn.execute(text(query_inner_join))
print("INNER JOIN:")
for row in result:
    print(row)

INNER JOIN:
('Чебурашка', 'Эдуард Успенский', 'Детская литература')
('Война и мир', 'Лев Толстой', 'Классическая литература')
('Анна Каренина', 'Лев Толстой', 'Классическая литература')
('Евгений Онегин', 'Александр Пушкин', 'Поэзия')
('Преступление и наказание', 'Фёдор Достоевский', 'Классическая литература')
('Бесы', 'Фёдор Достоевский', 'Классическая литература')


In [452]:
# JOIN (LEFT)

query_left_join = """
SELECT books.title, authors.name, categories.name
FROM books
LEFT JOIN authors ON books.author_id = authors.id
LEFT JOIN book_categories ON books.id = book_categories.book_id
LEFT JOIN categories ON book_categories.category_id = categories.id
"""

In [453]:
result = conn.execute(text(query_left_join))
print("LEFT JOIN:")
for row in result:
    print(row)

LEFT JOIN:
('Чебурашка', 'Эдуард Успенский', 'Детская литература')
('Война и мир', 'Лев Толстой', 'Классическая литература')
('Анна Каренина', 'Лев Толстой', 'Классическая литература')
('Евгений Онегин', 'Александр Пушкин', 'Поэзия')
('Преступление и наказание', 'Фёдор Достоевский', 'Классическая литература')
('Бесы', 'Фёдор Достоевский', 'Классическая литература')
('Приключения Тома Сойера', 'Марк Твен', None)


In [454]:
# UNION

query_union = """
SELECT title AS name 
FROM books
UNION
SELECT name FROM authors
"""

In [455]:
result = conn.execute(text(query_union))
print("UNION:")
for row in result:
    print(row)

UNION:
('Александр Пушкин',)
('Анна Каренина',)
('Бесы',)
('Война и мир',)
('Евгений Онегин',)
('Лев Толстой',)
('Марк Твен',)
('Преступление и наказание',)
('Приключения Тома Сойера',)
('Фёдор Достоевский',)
('Чебурашка',)
('Эдуард Успенский',)


In [456]:
# Подзапрос

query_subquery = """
SELECT title, (
    SELECT name
    FROM authors
    WHERE authors.id = books.author_id
) AS author_name
FROM books
"""

In [457]:
result = conn.execute(text(query_subquery))
print("Подзапрос:")
for row in result:
    print(row)

Подзапрос:
('Чебурашка', 'Эдуард Успенский')
('Война и мир', 'Лев Толстой')
('Анна Каренина', 'Лев Толстой')
('Евгений Онегин', 'Александр Пушкин')
('Преступление и наказание', 'Фёдор Достоевский')
('Бесы', 'Фёдор Достоевский')
('Приключения Тома Сойера', 'Марк Твен')


In [598]:
conn.close()
engine.dispose()

In [599]:
import os
os.remove("library.db")

In [600]:
engine = db.create_engine('sqlite:///library.db')
conn = engine.connect()
metadata = db.MetaData()

In [601]:
authors = db.Table("authors",metadata,
                   db.Column("id", db.INTEGER, primary_key=True),
                   db.Column("name", db.TEXT, nullable=False))

In [602]:
books = db.Table("books",metadata,
                 db.Column("id", db.INTEGER, primary_key=True),
                 db.Column("title", db.TEXT, nullable=False),
                 db.Column("author_id", db.INTEGER),
                 db.Column("price", db.REAL))

In [603]:
categories = db.Table("categories",metadata,
                      db.Column("id", db.INTEGER, primary_key=True),
                      db.Column("name", db.TEXT, nullable=False))

In [604]:
book_categories = db.Table("book_categories",metadata,
                           db.Column("book_id", db.INTEGER, db.ForeignKey("books.id")),
                           db.Column("category_id", db.INTEGER, db.ForeignKey("categories.id"), nullable=True),
                           db.PrimaryKeyConstraint("book_id", "category_id"))

In [605]:
metadata.create_all(engine)

In [606]:
insertions = []

In [608]:
insertion = authors.insert().values([
    {"name": "Эдуард Успенский"},
    {"name": "Лев Толстой"},
    {"name": "Марк Твен"},
    {"name": "Фёдор Достоевский"},
    {"name": "Александр Пушкин"}
])
insertions.append(insertion)

In [609]:
insertion = books.insert().values([
    {"title": "Чебурашка", "author_id": 1, "price": 15.0},
    {"title": "Война и мир", "author_id": 2, "price": 30.0},
    {"title": "Анна Каренина", "author_id": 2, "price": 25.0},
    {"title": "Евгений Онегин", "author_id": 5, "price": 20.0},
    {"title": "Преступление и наказание", "author_id": 4, "price": 22.0},
    {"title": "Бесы", "author_id": 4, "price": 24.0},
    {"title": "Приключения Тома Сойера", "author_id": 3, "price": 19.0}
])
insertions.append(insertion)

In [610]:
insertion = categories.insert().values([
    {"name": "Детская литература"},
    {"name": "Классическая литература"},
    {"name": "Поэзия"},
    {"name": "Роман"}
])
insertions.append(insertion)

In [611]:
insertion = book_categories.insert().values([
    {"book_id": 1, "category_id": 1},
    {"book_id": 2, "category_id": 2},
    {"book_id": 3, "category_id": 2},
    {"book_id": 4, "category_id": 3},
    {"book_id": 5, "category_id": 2},
    {"book_id": 6, "category_id": 2},
    {"book_id": 7, "category_id": db.null()},
])
insertions.append(insertion)

In [612]:
for insertion in insertions:
    conn.execute(insertion)

In [613]:
# HAVING

having = db.select(authors.c.name,
                   db.func.count(books.c.id).label('book_count'))\
            .select_from(authors.join(books, authors.c.id == books.c.author_id))\
            .group_by(authors.c.name)\
            .having(db.func.count(books.c.id) > 1)

In [614]:
result = conn.execute(having)
print("HAVING:")
for row in result:
    print(row)

HAVING:
('Лев Толстой', 2)
('Фёдор Достоевский', 2)


In [615]:
# JOIN (INNER)

inner_join = db.select(books.c.title,
                       authors.c.name,
                       categories.c.name)\
                .select_from(
                    books.join(authors, books.c.author_id == authors.c.id)\
                        .join(book_categories, books.c.id == book_categories.c.book_id)\
                        .join(categories, book_categories.c.category_id == categories.c.id)
                )

In [616]:
result = conn.execute(inner_join)
print("INNER JOIN:")
for row in result:
    print(row)

INNER JOIN:
('Чебурашка', 'Эдуард Успенский', 'Детская литература')
('Война и мир', 'Лев Толстой', 'Классическая литература')
('Анна Каренина', 'Лев Толстой', 'Классическая литература')
('Евгений Онегин', 'Александр Пушкин', 'Поэзия')
('Преступление и наказание', 'Фёдор Достоевский', 'Классическая литература')
('Бесы', 'Фёдор Достоевский', 'Классическая литература')


In [661]:
# JOIN (LEFT)

left_join = db.select(books.c.title,
                      authors.c.name,
                      categories.c.name)\
            .select_from(
                books.outerjoin(authors, books.c.author_id == authors.c.id)\
                    .outerjoin(book_categories, books.c.id == book_categories.c.book_id)\
                    .outerjoin(categories, book_categories.c.category_id == categories.c.id)
            )

In [618]:
result = conn.execute(left_join)
print("LEFT JOIN:")
for row in result:
    print(row)

LEFT JOIN:
('Чебурашка', 'Эдуард Успенский', 'Детская литература')
('Война и мир', 'Лев Толстой', 'Классическая литература')
('Анна Каренина', 'Лев Толстой', 'Классическая литература')
('Евгений Онегин', 'Александр Пушкин', 'Поэзия')
('Преступление и наказание', 'Фёдор Достоевский', 'Классическая литература')
('Бесы', 'Фёдор Достоевский', 'Классическая литература')
('Приключения Тома Сойера', 'Марк Твен', None)


In [622]:
# UNION
"""
SELECT title AS name 
FROM books
UNION
SELECT name FROM authors
"""

query_1 = db.select(books.c.title)
query_2 = db.select(authors.c.name)
union = db.union(query_1, query_2)

In [623]:
result = conn.execute(union)
print("UNION:")
for row in result:
    print(row)

UNION:
('Александр Пушкин',)
('Анна Каренина',)
('Бесы',)
('Война и мир',)
('Евгений Онегин',)
('Лев Толстой',)
('Марк Твен',)
('Преступление и наказание',)
('Приключения Тома Сойера',)
('Фёдор Достоевский',)
('Чебурашка',)
('Эдуард Успенский',)


In [655]:
# Подзапрос
"""
SELECT title, (
    SELECT name
    FROM authors
    WHERE authors.id = books.author_id
) AS author_name
FROM books
"""

query_1 = db.select(authors.c.name)\
            .select_from(authors)\
            .where(authors.c.id == books.c.author_id)
query_2 = db.select(books.c.title, query_1.label('author_name'))

In [656]:
result = conn.execute(query_2)
print("Подзапрос:")
for row in result:
    print(row)

Подзапрос:
('Чебурашка', 'Эдуард Успенский')
('Война и мир', 'Лев Толстой')
('Анна Каренина', 'Лев Толстой')
('Евгений Онегин', 'Александр Пушкин')
('Преступление и наказание', 'Фёдор Достоевский')
('Бесы', 'Фёдор Достоевский')
('Приключения Тома Сойера', 'Марк Твен')
